In [10]:
#  ASR Notes 
# ==========================
#     1/ Store S&P symbols for ALL years in one File , so that easy to Load into dataFramme and slice it
#     2/ gains also store in One FILE for ALL Years , so that we can plot and compare multiple yeras
#     3/ Yahoo API give errors or NO DATA Found once a TICKER is delisted
#       example: ALTR company bought in 2016, but still from 2008 onward NODATA/Errors 
#       we may need to find a way , to get this OLD DATA, else we are missing 5% to 10% every year
#
# StackOverFlow good links:
# =============================
# 1/ DF Copy:  http://stackoverflow.com/questions/34682828/pandas-extracting-specific-selected-columns-from-a-dataframe-to-new-dataframe   
  
# 2/ Adding a Row : http://stackoverflow.com/questions/27873190/creating-new-pandas-dataframe-from-existing-dataframe-and-index
        

In [11]:
#import desired packages
import numpy as np
import pandas as pd
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
import time
%matplotlib inline 

In [12]:
import urllib.request

def get_year_open_close_price(symbol, year):
    u = "http://chart.finance.yahoo.com/table.csv?s={}&a={}&b={}&c={}&d={}&e={}&f={}&g=d&ignore=.csv"
    month, day = 1, 1  # JAN 1 , first day
    u1 = u.format(symbol, month-1, day, year, month-1, day+5, year)
    
    month, day = 12, 31  # Dec 31 , last day
    u2 = u.format(symbol, month-1, day-5, year, month-1, day, year)
    
    try:
        urllib.request.urlretrieve (u1, "mydat.csv")
        df = pd.read_csv('mydat.csv', header=0)
        # why -1 ? to get first record value by Date
        open_day1 = float(df['Open'].tolist()[-1])  
        high_day1 = float(df['High'].tolist()[-1])
        close_day1 = float(df['Close'].tolist()[-1])
    except: 
        print ('no Data for {} , for the year {}'.format(symbol, year))
        return []  # retrun empty in case of exception

    urllib.request.urlretrieve (u2, "mydat.csv")
    df = pd.read_csv('mydat.csv', header=0)
    close_daylast = float(df['Close'].tolist()[0])  # why 0 ? to get last record value ny Date
    
    return [ year,symbol,round(open_day1, 1),round(high_day1, 1),round(close_day1,1),round(close_daylast,1)]


In [13]:
for year in range(2008, 2009):
    rec = get_year_open_close_price('IBM', year)
    print (rec)

[2008, 'IBM', 109.0, 109.0, 104.7, 84.2]


In [15]:
#import SnP Symbol list 
snp_tickers_allyears = pd.read_csv('data-SnP-10years-Tickers-byyear-dfready.csv', header=0)

gains_lst = []
nodata_found_lst = []
for year in range(2010,2016):
     
    llst = snp_tickers_allyears.query('Year == @year').values.tolist()
    #llst = [ ['Ticker', 'Year'], ['AAPL', 2008]]
    for stock in llst[1:] :  # skip 0 th element being it header
        # here index are : 0 - TIcker , 1 - year
        #print('trying to fetch data for ' + stock[0])
        try:
            rec = get_year_open_close_price(stock[0] , stock[1])  
            if len(rec) > 0:
                gains_lst.append(rec)
                #print( rec[1] + ' is good')
            else:
                print(stock[0])
                nodata_found_lst.append(stock)
        except:
            nodata_found_lst.append(stock)
            #print('Exception for { }, year {}'.format(stock[0], stock[1])

df = pd.DataFrame(gains_lst)
cols = ['Year', 'Ticker', 'Open_day1', 'High_day1', 'Close_day1', 'Close_daylast']
df.columns = cols
df['Gain_yearly'] = round( ((df['Close_daylast'] - df['Open_day1']) * 100 )/df['Open_day1'], 1) 

df2 = df.sort_values(by='Gain_yearly', ascending=False)
df2.to_csv('data-SnP-yearly-performance.csv', index=False)
nodata_df = pd.DataFrame(nodata_found_lst)
nodata_df.columns = ['Ticker', 'Year']
nodata_df.to_csv('data-nodata-found-tickers.csv', index=False)


no Data for ALTR , for the year 2010
ALTR
no Data for AYE , for the year 2010
AYE
no Data for BDK , for the year 2010
BDK
no Data for BF.B , for the year 2010
BF.B
no Data for BJS , for the year 2010
BJS
no Data for BNI , for the year 2010
BNI
no Data for BRCM , for the year 2010
BRCM
no Data for BTU , for the year 2010
BTU
no Data for CAM , for the year 2010
CAM
no Data for CEPH , for the year 2010
CEPH
no Data for DTV , for the year 2010
DTV
no Data for EK , for the year 2010
EK
no Data for EP , for the year 2010
EP
no Data for FDO , for the year 2010
FDO
no Data for GAS , for the year 2010
GAS
no Data for GCI , for the year 2010
GCI
no Data for GENZ , for the year 2010
GENZ
no Data for HCBK , for the year 2010
HCBK
no Data for HSP , for the year 2010
HSP
no Data for IMS , for the year 2010
IMS
no Data for JAVA , for the year 2010
JAVA
no Data for KG , for the year 2010
KG
no Data for LIFE , for the year 2010
LIFE
no Data for LO , for the year 2010
LO
no Data for MFE , for the year 2

In [5]:
df = pd.read_csv('data-SnP-yearly-performance.csv', header=0)

df2 = df.sort_values(by='Gain_yearly', ascending=False) 
df3 = df2.query('Gain_yearly > 80 or Gain_yearly  < -40')
df3

,Year,Ticker,Open_day1,High_day1,Close_day1,Close_daylast,Gain_yearly
0,2009,GR,0.0,0.0,0.0,0.1,inf
1,2009,AIG,1.6,1.7,1.7,30.0,1775.000000
2,2009,XL,3.8,4.0,3.9,18.3,381.578947
3,2009,THC,1.2,1.3,1.3,5.4,350.000000
4,2009,AMD,2.2,2.4,2.4,9.7,340.909091
5,2009,F,2.3,2.5,2.5,10.0,334.782609
6,2009,MU,2.6,2.9,2.8,10.6,307.692308
7,2009,GNW,2.9,2.9,2.9,11.3,289.655172
8,2009,FCX,24.9,27.0,26.7,80.3,222.489960
9,2009,EXPE,8.2,8.8,8.8,25.7,213.414634
